# 제주 재난지원금 데이터 기본 분석


기본 임포트

In [1]:
import numpy as np
import pandas as pd

데이터 불러오기 (필요한 데이터가 있으면 여기에 추가)

In [2]:
# data_05 = pd.read_csv("./data/KRI-DAC_Jeju_data5.txt")
data_06 = pd.read_csv("./data/KRI-DAC_Jeju_data6.txt")

In [3]:
data_06.head()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
0,1,1,202006,제주특별자치도,제주시,영세,일반한식,00시,502000,0,10,0,877005.9834,1.479766e+06
1,2,2,202006,제주특별자치도,제주시,영세,단란주점,00시,1520000,0,8,0,877005.7447,1.479816e+06
2,3,3,202006,제주특별자치도,제주시,중소1,편의점,00시,482310,0,35,0,877056.6756,1.479616e+06
3,4,4,202006,제주특별자치도,제주시,영세,편의점,00시,38050,5450,3,1,877055.9593,1.479766e+06
4,5,5,202006,제주특별자치도,제주시,영세,일반한식,00시,32000,32000,1,1,877055.4817,1.479866e+06


In [4]:
data = pd.concat([data_06])
data = data.reset_index(drop=True)
data.head()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
0,1,1,202006,제주특별자치도,제주시,영세,일반한식,00시,502000,0,10,0,877005.9834,1.479766e+06
1,2,2,202006,제주특별자치도,제주시,영세,단란주점,00시,1520000,0,8,0,877005.7447,1.479816e+06
2,3,3,202006,제주특별자치도,제주시,중소1,편의점,00시,482310,0,35,0,877056.6756,1.479616e+06
3,4,4,202006,제주특별자치도,제주시,영세,편의점,00시,38050,5450,3,1,877055.9593,1.479766e+06
4,5,5,202006,제주특별자치도,제주시,영세,일반한식,00시,32000,32000,1,1,877055.4817,1.479866e+06


좌표계 변환

In [5]:
from pyproj import Proj, transform
import geopandas as gpd

def get_WGS84(coordinates):
    """
    UTM-K 좌표계를 WGS로 변환해주는 함수.
    coordinates: UTM-K 좌표 리스트
    """
    proj_WGS84 = Proj(init="epsg:4326") # WGS1984
    proj_UTMK = Proj('+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 + units=m +no_defs')  # UTM-K 좌표계 변환 공식
    
    long = [x[0] for x in coordinates]  # UTM-K 경도
    lat = [x[1] for x in coordinates]   # UTM-K 위도
    
    coordinates_w84 = transform(proj_UTMK, proj_WGS84, long, lat)
    
    return coordinates_w84

In [6]:
coords_utmk = list(zip(list(data.POINT_X), list(data.POINT_Y)))

# 좌표 변환
coords_wgs84 = get_WGS84(coords_utmk)

# 데이터 프레임 좌표값 수정
data["POINT_X"] = coords_wgs84[0]
data["POINT_Y"] = coords_wgs84[1]
data.head()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
0,1,1,202006,제주특별자치도,제주시,영세,일반한식,00시,502000,0,10,0,126.178894,33.302315
1,2,2,202006,제주특별자치도,제주시,영세,단란주점,00시,1520000,0,8,0,126.178884,33.302766
2,3,3,202006,제주특별자치도,제주시,중소1,편의점,00시,482310,0,35,0,126.179458,33.300971
3,4,4,202006,제주특별자치도,제주시,영세,편의점,00시,38050,5450,3,1,126.179430,33.302323
4,5,5,202006,제주특별자치도,제주시,영세,일반한식,00시,32000,32000,1,1,126.179412,33.303224


읍면동 컬럼 추가 with kakao api

In [7]:
data["DONG"] = ""
data.head()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,DONG
0,1,1,202006,제주특별자치도,제주시,영세,일반한식,00시,502000,0,10,0,126.178894,33.302315,
1,2,2,202006,제주특별자치도,제주시,영세,단란주점,00시,1520000,0,8,0,126.178884,33.302766,
2,3,3,202006,제주특별자치도,제주시,중소1,편의점,00시,482310,0,35,0,126.179458,33.300971,
3,4,4,202006,제주특별자치도,제주시,영세,편의점,00시,38050,5450,3,1,126.179430,33.302323,
4,5,5,202006,제주특별자치도,제주시,영세,일반한식,00시,32000,32000,1,1,126.179412,33.303224,


In [8]:
kakao_access_key = "e0f925a97415f0a86406eab7105e2e2a"

임시로 api 테스트 진행

In [9]:
import requests
import json
url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=126.178894&y=33.302315"

headers = {"Authorization": "KakaoAK {}".format(kakao_access_key)}
api_test = requests.get(url,headers=headers)
url_text = json.loads(api_test.text)
url_text

{'meta': {'total_count': 2},
 'documents': [{'region_type': 'B',
   'code': '5011031023',
   'address_name': '제주특별자치도 제주시 한경면 고산리',
   'region_1depth_name': '제주특별자치도',
   'region_2depth_name': '제주시',
   'region_3depth_name': '한경면',
   'region_4depth_name': '고산리',
   'x': 126.18203028380402,
   'y': 33.29722725544281},
  {'region_type': 'H',
   'code': '5011031000',
   'address_name': '제주특별자치도 제주시 한경면',
   'region_1depth_name': '제주특별자치도',
   'region_2depth_name': '제주시',
   'region_3depth_name': '한경면',
   'region_4depth_name': '',
   'x': 126.18415707698696,
   'y': 33.35013231995938}]}

In [10]:
data.shape

(281896, 15)

In [11]:
data = data[189000:281759]
data

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,DONG
189000,189001,189001,202006,제주특별자치도,서귀포시,영세,일반한식,17시,185000,0,2,0,126.934951,33.464367,
189001,189002,189002,202006,제주특별자치도,서귀포시,일반,일반한식,17시,241000,0,2,0,126.934946,33.464818,
189002,189003,189003,202006,제주특별자치도,서귀포시,영세,일반한식,17시,24000,0,1,0,126.934940,33.465269,
189003,189004,189004,202006,제주특별자치도,서귀포시,영세,기타숙박업,17시,36000,0,2,0,126.934935,33.465719,
189004,189005,189005,202006,제주특별자치도,서귀포시,중소2,부동산중개임대,17시,1211800,16000,59,1,126.934896,33.468875,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281754,281755,281755,202006,제주특별자치도,서귀포시,중소,일반한식,x시,545600,0,7,0,126.919633,33.441694,
281755,281756,281756,202006,제주특별자치도,서귀포시,영세,일반한식,x시,60000,0,2,0,126.919298,33.468740,
281756,281757,281757,202006,제주특별자치도,서귀포시,영세,스넥,x시,9500,0,1,0,126.920238,33.436289,
281757,281758,281758,202006,제주특별자치도,서귀포시,일반,일반한식,x시,34800,0,1,0,126.920238,33.436289,


In [12]:
import time

headers = {"Authorization": "KakaoAK {}".format(kakao_access_key)}

start = time.time()
count = 0

for i, j in data.iterrows():
    POINT_X = j.POINT_X
    POINT_Y = j.POINT_Y
    url = ('https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(POINT_X, POINT_Y))
    api_test = requests.get(url,headers=headers)
    url_text = json.loads(api_test.text)
    data["DONG"][i] = url_text["documents"][0]["region_3depth_name"]
    count += 1
    if (count % 10000 == 0):
        print("progress {}/90000".format(count))
    
print("Finish convert", time.time()-start)

<ipython-input-12-a29275a59a6c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["DONG"][i] = url_text["documents"][0]["region_3depth_name"]


progress 10000/90000
progress 20000/90000
progress 30000/90000
progress 40000/90000
progress 50000/90000
progress 60000/90000
progress 70000/90000
progress 80000/90000
progress 90000/90000
Finish convert 7591.70719909668


In [13]:
data.shape

(92759, 15)

In [14]:
data.head()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,DONG
189000,189001,189001,202006,제주특별자치도,서귀포시,영세,일반한식,17시,185000,0,2,0,126.934951,33.464367,성산읍
189001,189002,189002,202006,제주특별자치도,서귀포시,일반,일반한식,17시,241000,0,2,0,126.934946,33.464818,성산읍
189002,189003,189003,202006,제주특별자치도,서귀포시,영세,일반한식,17시,24000,0,1,0,126.934940,33.465269,성산읍
189003,189004,189004,202006,제주특별자치도,서귀포시,영세,기타숙박업,17시,36000,0,2,0,126.934935,33.465719,성산읍
189004,189005,189005,202006,제주특별자치도,서귀포시,중소2,부동산중개임대,17시,1211800,16000,59,1,126.934896,33.468875,성산읍


In [15]:
data.to_csv("./data/KRI-DAC_Jeju_data6_dong_189000_281759.csv")
print("PROCESSING FINISHED!!!")

PROCESSING FINISHED!!!
